In [2]:
from typing import Dict, List
import wandb
import pandas as pd
import numpy as np

api = wandb.Api()

In [4]:
experiments_list = pd.read_csv("../../Paper/experiments_paper.csv")

In [ ]:
experiments_list

In [ ]:
def to_camel_case(snake_str):
    components = snake_str.split('_')
    return components[0] + ''.join(x.title() for x in components[1:])

def number_to_word(number: int):
    num_to_word = {
        1: "one",
        5: "five",
        10: "ten",
        15: "fiveteen",
        20: "twenty",
        50: "fifty"
    }

    return num_to_word.get(number, number)

def retrieve_table(experiments_list: pd.DataFrame, output_file_name: str, columns: List[str], iterations: List[int]) -> None:
    experiments = experiments_list["Strategie"].unique()

    metrics = {"Strategy": [], "Interpolation": [], "Block Size": []}

    for column in columns:
        for iteration in iterations:
            metrics[f"{column}_mean_{number_to_word(iteration)}"] = []
            metrics[f"{column}_std_{number_to_word(iteration)}"] = []

    for experiment in experiments:
        print("experiment:", experiment)
        runs = experiments_list[experiments_list["Strategie"] == experiment]

        assert len(runs) == 3

        experiment_metrics = {column: {iteration: [] for iteration in iterations} for column in columns}

        for _, runs in runs.iterrows():
            run_url = runs["Wandb Link"].replace("https://wandb.ai", "")

            run = api.run(run_url)

            for column in columns:
                full_column_name = f"best_model_val/{column}"

                if full_column_name not in run.summary:
                    print(f"Column {full_column_name} does not existing for dataset.")

                metric_history = [item for item in run.scan_history(keys=[full_column_name, "trainer/iteration"])]

                metric_history = pd.DataFrame(metric_history)

                for iteration in iterations:
                    metric_value = metric_history.loc[metric_history["trainer/iteration"] == iteration, full_column_name]
                    assert len(metric_value) == 1
                    experiment_metrics[column][iteration].append(metric_value.iloc[0])


        descriptions = runs["Description"].split(", ")

        metrics["Strategy"].append(descriptions[0])

        if len(descriptions) > 1:
            metrics["Interpolation"].append(descriptions[1].capitalize())
            metrics["Block Size"].append(descriptions[2])
        else:
            metrics["Interpolation"].append(float("nan"))
            metrics["Block Size"].append(float("nan"))

        for column in columns:
            for iteration in iterations:
                current_metrics = np.array(experiment_metrics[column][iteration])
                metrics[f"{column}_mean_{number_to_word(iteration)}"].append(current_metrics.mean().round(decimals=3))
                metrics[f"{column}_std_{number_to_word(iteration)}"].append(current_metrics.std(ddof=1).round(decimals=3))

    metrics = pd.DataFrame(metrics)
    metrics = metrics.rename({column: to_camel_case(column.replace(" ", "_")) for column in metrics.columns}, axis="columns")
    metrics.to_csv(output_file_name, index=False)

In [ ]:
columns = ["mean_dice_score"]
iterations = [5, 10, 15, 20, 50]

experiments = [
                "Random Sampling",
                "Uncertainty Sampling (Entropy)",
                "Uncertainty Sampling (Distance)",
                "Representativeness Sampling (Uniform Sampling of Slices from all Scans)",
                "Representativeness Sampling (Mean-Shift Clustering of Feature Vectors)",
                "Representativeness Sampling (Maximum Distance of Feature Vectors)"
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Hippocampus"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

retrieve_table(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/hippocampus/baseline_strategies.csv",
                columns,
                iterations)

In [ ]:
columns = ["mean_dice_score"]
iterations = [5, 10, 15, 20, 50]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 5",
                "Signed Distance Interpolation, Random, Block Size 5",
                "ITK Interpolation, Random, Block Size 5",
                "Block Selection ohne Interpolation, Random, Block Size 10",
                "Signed Distance Interpolation, Random, Block Size 10",
                "ITK Interpolation, Random, Block Size 10",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Hippocampus"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

retrieve_table(current_experiment_list,
                "/Users/josafat/1-Documents/4_Studium/IT-Systems-Engineering/2_Skripte/2-Master/3._Semester/Masterprojekt/Paper/Latex/data/hippocampus/interpolation_strategies_random.csv",
                columns, iterations)

In [ ]:
columns = ["mean_dice_score"]
iterations = [5, 10, 15, 20, 50]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Uncertainty (Entropy), Block Size 5",
                "Signed Distance Interpolation, Uncertainty (Entropy), Block Size 5",
                "ITK Interpolation, Uncertainty (Entropy), Block Size 5",
                "Block Selection ohne Interpolation, Uncertainty (Entropy), Block Size 10",
                "Signed Distance Interpolation, Uncertainty (Entropy), Block Size 10",
                "ITK Interpolation, Uncertainty (Entropy), Block Size 10",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Hippocampus"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

retrieve_table(current_experiment_list,
                "/Users/josafat/1-Documents/4_Studium/IT-Systems-Engineering/2_Skripte/2-Master/3._Semester/Masterprojekt/Paper/Latex/data/hippocampus/interpolation_strategies_uncertainty.csv",
                columns, iterations)

In [ ]:
columns = ["mean_dice_score"]
iterations = [5, 10, 15, 20, 50]

experiments = [
                "Random Sampling",
                "Uncertainty Sampling (Entropy)",
                "Uncertainty Sampling (Distance)",
                "Representativeness Sampling (Uniform Sampling of Slices from all Scans)",
                "Representativeness Sampling (Mean-Shift Clustering of Feature Vectors)",
                "Representativeness Sampling (Maximum Distance of Feature Vectors)"
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Heart"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

retrieve_table(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/heart/baseline_strategies.csv",
                columns,
                iterations)

In [ ]:
columns = ["mean_dice_score"]
iterations = [5, 10, 15, 20, 50]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 5",
                "Signed Distance Interpolation, Random, Block Size 5",
                "ITK Interpolation, Random, Block Size 5",
                "Block Selection ohne Interpolation, Random, Block Size 10",
                "Signed Distance Interpolation, Random, Block Size 10",
                "ITK Interpolation, Random, Block Size 10",
                "Block Selection ohne Interpolation, Random, Block Size 15",
                "Signed Distance Interpolation, Random, Block Size 15",
                "ITK Interpolation, Random, Block Size 15",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Heart"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

retrieve_table(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/heart/interpolation_strategies_random.csv",
                columns, iterations)

In [ ]:
columns = ["mean_dice_score"]
iterations = [5, 10, 15, 20, 50]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Uncertainty (Entropy), Block Size 5",
                "Signed Distance Interpolation, Uncertainty (Entropy), Block Size 5",
                "ITK Interpolation, Uncertainty (Entropy), Block Size 5",
                "Block Selection ohne Interpolation, Uncertainty (Entropy), Block Size 10",
                "Signed Distance Interpolation, Uncertainty (Entropy), Block Size 10",
                "ITK Interpolation, Uncertainty (Entropy), Block Size 10",
                "Block Selection ohne Interpolation, Uncertainty (Entropy), Block Size 15",
                "Signed Distance Interpolation, Uncertainty (Entropy), Block Size 15",
                "ITK Interpolation, Uncertainty (Entropy), Block Size 15",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Heart"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

retrieve_table(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/heart/interpolation_strategies_uncertainty.csv",
                columns, iterations)

In [ ]:
columns = ["mean_dice_score"]
iterations = [5, 10, 15, 20, 50]

experiments = [
                "Random Sampling",
                "Uncertainty Sampling (Entropy)",
                "Uncertainty Sampling (Distance)",
                "Representativeness Sampling (Uniform Sampling of Slices from all Scans)",
                "Representativeness Sampling (Mean-Shift Clustering of Feature Vectors)",
                "Representativeness Sampling (Maximum Distance of Feature Vectors)"
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Prostate"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

retrieve_table(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/prostate/baseline_strategies.csv",
                columns,
                iterations)

In [ ]:
columns = ["mean_dice_score"]
iterations = [5, 10, 15, 20, 50]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 5",
                "Signed Distance Interpolation, Random, Block Size 5",
                "ITK Interpolation, Random, Block Size 5",
                "Block Selection ohne Interpolation, Random, Block Size 10",
                "Signed Distance Interpolation, Random, Block Size 10",
                "ITK Interpolation, Random, Block Size 10",
                "Block Selection ohne Interpolation, Random, Block Size 15",
                "Signed Distance Interpolation, Random, Block Size 15",
                "ITK Interpolation, Random, Block Size 15",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Prostate"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

retrieve_table(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/prostate/interpolation_strategies_random.csv",
                columns, iterations)

In [ ]:
columns = ["mean_dice_score"]
iterations = [5, 10, 15, 20, 50]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Uncertainty (Entropy), Block Size 5",
                "Signed Distance Interpolation, Uncertainty (Entropy), Block Size 5",
                "ITK Interpolation, Uncertainty (Entropy), Block Size 5",
                "Block Selection ohne Interpolation, Uncertainty (Entropy), Block Size 10",
                "Signed Distance Interpolation, Uncertainty (Entropy), Block Size 10",
                "ITK Interpolation, Uncertainty (Entropy), Block Size 10",
                "Block Selection ohne Interpolation, Uncertainty (Entropy), Block Size 15",
                "Signed Distance Interpolation, Uncertainty (Entropy), Block Size 15",
                "ITK Interpolation, Uncertainty (Entropy), Block Size 15",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Prostate"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

retrieve_table(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/prostate/interpolation_strategies_uncertainty.csv",
                columns, iterations)

### Diagrams

In [ ]:
def get_diagram_data(experiment_list, output_file_path: str, columns, iterations: int = 50):
    experiment_data = {}
    experiment_data["iteration"] = list(range(1, iterations+2))

    experiments = experiment_list["Strategie"].unique()

    for experiment in experiments:
        current_experiments = experiment_list[experiment_list["Strategie"] == experiment]

        current_metrics = {column: np.zeros((iterations + 1, len(current_experiments))) for column in columns}
        
        for idx, (_, experiment) in enumerate(current_experiments.iterrows()):
            run_url = experiment["Wandb Link"].replace("https://wandb.ai", "")
            experiment_name = experiment["Strategie"]

            run = api.run(run_url)

            for column in columns:
                full_column_name = f"best_model_val/{column}"

                if full_column_name not in run.summary:
                    print(f"Column {full_column_name} does not existing for dataset.")

                metric_history = [item for item in run.scan_history(keys=[full_column_name, "trainer/iteration"])]
                metric_history = pd.DataFrame(metric_history)

                if len(metric_history) == iterations + 1:
                    current_metrics[column][:, idx] = metric_history[full_column_name].dropna().values
                else:
                    print(f"{run_url} is incomplete.")

        for column in columns:
            experiment_data[f"{experiment_name.replace(', ', '_')}_{column.replace('bestModelVal', '')}_mean"] = current_metrics[column].mean(axis=1)
            experiment_data[f"{experiment_name.replace(', ', '_')}_{column.replace('bestModelVal', '')}_min"] = current_metrics[column].min(axis=1)
            experiment_data[f"{experiment_name.replace(', ', '_')}_{column.replace('bestModelVal', '')}_max"] = current_metrics[column].max(axis=1)

    experiment_data = pd.DataFrame(experiment_data)
    experiment_data.to_csv(output_file_path, index=False)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Uncertainty Sampling (Entropy)",
                "Uncertainty Sampling (Distance)",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Heart"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/heart/diagram_uncertainty_sampling.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Uncertainty Sampling (Entropy)",
                "Uncertainty Sampling (Distance)",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Hippocampus"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/hippocampus/diagram_uncertainty_sampling.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Uncertainty Sampling (Entropy)",
                "Uncertainty Sampling (Distance)",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Prostate"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/prostate/diagram_uncertainty_sampling.csv",
                columns,
                50)

### Representativeness Sampling

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Representativeness Sampling (Uniform Sampling of Slices from all Scans)",
                "Representativeness Sampling (Mean-Shift Clustering of Feature Vectors)",
                "Representativeness Sampling (Maximum Distance of Feature Vectors)",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Heart"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/heart/diagram_representativeness_sampling.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Representativeness Sampling (Uniform Sampling of Slices from all Scans)",
                "Representativeness Sampling (Mean-Shift Clustering of Feature Vectors)",
                "Representativeness Sampling (Maximum Distance of Feature Vectors)",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Hippocampus"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/hippocampus/diagram_representativeness_sampling.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Representativeness Sampling (Uniform Sampling of Slices from all Scans)",
                "Representativeness Sampling (Mean-Shift Clustering of Feature Vectors)",
                "Representativeness Sampling (Maximum Distance of Feature Vectors)",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Prostate"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/prostate/diagram_representativeness_sampling.csv",
                columns,
                50)

### Block Query Strategies

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 5",
                "Signed Distance Interpolation, Random, Block Size 5",
                "ITK Interpolation, Random, Block Size 5",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Heart"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/1-Documents/4_Studium/IT-Systems-Engineering/2_Skripte/2-Master/3._Semester/Masterprojekt/Paper/Latex/data/heart/diagram_interpolation_random_block_size_5.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 5",
                "Signed Distance Interpolation, Random, Block Size 5",
                "ITK Interpolation, Random, Block Size 5",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Hippocampus"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/1-Documents/4_Studium/IT-Systems-Engineering/2_Skripte/2-Master/3._Semester/Masterprojekt/Paper/Latex/data/hippocampus/diagram_interpolation_random_block_size_5.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 5",
                "Signed Distance Interpolation, Random, Block Size 5",
                "ITK Interpolation, Random, Block Size 5",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Prostate"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/1-Documents/4_Studium/IT-Systems-Engineering/2_Skripte/2-Master/3._Semester/Masterprojekt/Paper/Latex/data/prostate/diagram_interpolation_random_block_size_5.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 10",
                "Signed Distance Interpolation, Random, Block Size 10",
                "ITK Interpolation, Random, Block Size 10",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Heart"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/heart/diagram_interpolation_random.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 10",
                "Signed Distance Interpolation, Random, Block Size 10",
                "ITK Interpolation, Random, Block Size 10",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Hippocampus"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/hippocampus/diagram_interpolation_random.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 10",
                "Signed Distance Interpolation, Random, Block Size 10",
                "ITK Interpolation, Random, Block Size 10",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Prostate"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/prostate/diagram_interpolation_random.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 15",
                "Signed Distance Interpolation, Random, Block Size 15",
                "ITK Interpolation, Random, Block Size 15",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Heart"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/1-Documents/4_Studium/IT-Systems-Engineering/2_Skripte/2-Master/3._Semester/Masterprojekt/Paper/Latex/data/heart/diagram_interpolation_random_block_size_15.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 15",
                "Signed Distance Interpolation, Random, Block Size 15",
                "ITK Interpolation, Random, Block Size 15",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Hippocampus"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/1-Documents/4_Studium/IT-Systems-Engineering/2_Skripte/2-Master/3._Semester/Masterprojekt/Paper/Latex/data/hippocampus/diagram_interpolation_random_block_size_15.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 15",
                "Signed Distance Interpolation, Random, Block Size 15",
                "ITK Interpolation, Random, Block Size 15",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Prostate"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/1-Documents/4_Studium/IT-Systems-Engineering/2_Skripte/2-Master/3._Semester/Masterprojekt/Paper/Latex/data/prostate/diagram_interpolation_random_block_size_15.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 10",
                "Signed Distance Interpolation, Random, Block Size 10",
                "ITK Interpolation, Random, Block Size 10",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Prostate"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/prostate/diagram_interpolation_random.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Uncertainty (Entropy), Block Size 10",
                "Signed Distance Interpolation, Uncertainty (Entropy), Block Size 10",
                "ITK Interpolation, Uncertainty (Entropy), Block Size 10",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Heart"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/heart/diagram_interpolation_uncertainty.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Uncertainty (Entropy), Block Size 10",
                "Signed Distance Interpolation, Uncertainty (Entropy), Block Size 10",
                "ITK Interpolation, Uncertainty (Entropy), Block Size 10",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Hippocampus"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/hippocampus/diagram_interpolation_uncertainty.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 5",
                "Block Selection ohne Interpolation, Random, Block Size 10",
                "Block Selection ohne Interpolation, Random, Block Size 15",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Hippocampus"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/1-Documents/4_Studium/IT-Systems-Engineering/2_Skripte/2-Master/3._Semester/Masterprojekt/Paper/Latex/data/hippocampus/diagram_block_selection.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 5",
                "Block Selection ohne Interpolation, Random, Block Size 10",
                "Block Selection ohne Interpolation, Random, Block Size 15",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Heart"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/1-Documents/4_Studium/IT-Systems-Engineering/2_Skripte/2-Master/3._Semester/Masterprojekt/Paper/Latex/data/heart/diagram_block_selection.csv",
                columns,
                50)

In [ ]:
columns = ["mean_dice_score"]

experiments = [
                "Random Sampling",
                "Block Selection ohne Interpolation, Random, Block Size 5",
                "Block Selection ohne Interpolation, Random, Block Size 10",
                "Block Selection ohne Interpolation, Random, Block Size 15",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Prostate"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

get_diagram_data(current_experiment_list,
                "/Users/josafat/1-Documents/4_Studium/IT-Systems-Engineering/2_Skripte/2-Master/3._Semester/Masterprojekt/Paper/Latex/data/prostate/diagram_block_selection.csv",
                columns,
                50)

### Interpolation Quality

In [ ]:
def retrieve_interpolation_metrics(experiments_list: pd.DataFrame, block_sizes: List[int], datasets: List[str]) -> Dict[str, np.ndarray]:
    interpolation_quality_metrics = {}
    interpolation_thickness = {}

    interpolation_names = {
        "ITK": "morphological contour interpolation",
        "Signed Distance": "signed distance interpolation"
    }

    experiments_list = experiments_list[experiments_list["Strategie"].str.contains("Random").fillna(False)]
    experiments_list = experiments_list[~experiments_list["Strategie"].str.contains("Block Selection ohne Interpolation").fillna(False)]

    for block_size in block_sizes:
        for dataset in datasets:
            current_experiments_list = experiments_list[experiments_list["Dataset"] == f"Decathlon, {dataset}"]
            current_experiments_list = current_experiments_list[current_experiments_list["Strategie"].str.contains(f"Block Size {block_size}").fillna(False)]

            for interpolation_method in ["ITK", "Signed Distance"]:
                current_experiments = current_experiments_list[current_experiments_list["Strategie"].str.contains(interpolation_method).fillna(False)]

                assert len(current_experiments) == 3

                if len(current_experiments) == 0:
                    print(f"{dataset} dataset, {interpolation_names[interpolation_method]}, block size {block_size}")

                current_metrics = []
                current_thickness = []

                for idx, (_, experiment) in enumerate(current_experiments.iterrows()):
                    run_url = experiment["Wandb Link"].replace("https://wandb.ai", "")

                    run = api.run(run_url)

                    for item in run.scan_history(keys=["val/mean_dice_score_interpolation", "val/interpolation_thickness"]):
                        if not type(item["val/mean_dice_score_interpolation"]) == str:
                            current_metrics.append(item["val/mean_dice_score_interpolation"])
                            current_thickness.append(item["val/interpolation_thickness"])

                current_metrics = np.array(current_metrics)
                current_thickness = np.array(current_thickness)

                if len(block_sizes) == 1:
                    key = f"{dataset} dataset, {interpolation_names[interpolation_method]}"
                else:
                    key = f"{dataset} dataset, {interpolation_names[interpolation_method]}, block size {block_size}"
        
                interpolation_quality_metrics[key] = current_metrics
                interpolation_thickness[key] = current_thickness

    return interpolation_quality_metrics, interpolation_thickness

In [ ]:
dice_scores, thickness = retrieve_interpolation_metrics(experiments_list, block_sizes=[10], datasets=["Heart", "Hippocampus", "Prostate"])

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

metrics_to_plot = [
                    "Hippocampus dataset, signed distance interpolation",
                    "Hippocampus dataset, morphological contour interpolation",
                    "Heart dataset, signed distance interpolation",
                    "Heart dataset, morphological contour interpolation",
                    "Prostate dataset, signed distance interpolation",
                    "Prostate dataset, morphological contour interpolation",
                    ]

curent_metrics = {key: value for key, value in dice_scores.items() if key in metrics_to_plot}

plt.boxplot(list(curent_metrics.values()), labels=list(curent_metrics.keys()), vert=False)

ax = plt.gca()
ax.set_xlabel("Mean interpolation Dice score")

plt.savefig("/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/images/interpolation_quality_datasets.pdf", bbox_inches = 'tight')

In [ ]:
plt.scatter(thickness["Hippocampus dataset, signed distance interpolation"], dice_scores["Hippocampus dataset, signed distance interpolation"])

In [ ]:
thickness_labels = []
thickness_values = []
column = "Prostate dataset, signed distance interpolation"

for block_thickness in np.unique(thickness[column]):
    thickness_labels.append(f"Block size {int(block_thickness)}")
    thickness_values.append(dice_scores[column][thickness[column] == block_thickness])

plt.boxplot(thickness_values, labels=thickness_labels, vert=False)

ax = plt.gca()
ax.set_xlabel("Mean interpolation Dice score")

plt.savefig("/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/images/interpolation_quality_thickness_prostate.pdf", bbox_inches = 'tight')

# plt.scatter(thickness["Heart dataset, signed distance interpolation"], dice_scores["Heart dataset, signed distance interpolation"])

In [ ]:
experiments = [
                  "Signed Distance Interpolation, Random, Block Size 5",
                  "ITK Interpolation, Random, Block Size 5",
                  "Signed Distance Interpolation, Random, Block Size 10",
                  "ITK Interpolation, Random, Block Size 10",
                  "Signed Distance Interpolation, Random, Block Size 15",
                  "ITK Interpolation, Random, Block Size 15",
            ]

current_experiment_list = experiments_list.copy()
current_experiment_list = current_experiment_list[current_experiment_list["Dataset"] == "Decathlon, Heart"]
current_experiment_list = current_experiment_list[current_experiment_list["Strategie"].isin(experiments)]

metrics = retrieve_interpolation_table(
      current_experiment_list,
      "/Users/josafat/Desktop/Semi-Supervised-Active-Learning-for-3D-Medical-Image-Segmentation---MDPI-Submission/data/interpolation_quality.csv"
      )
metrics

In [ ]:
experiments_list[experiments_list["Strategie"].str.contains("Block Size 5").fillna(False)]

In [ ]:
metrics